<a href="https://colab.research.google.com/github/eteitelbaum/code-satp/blob/Fall-2024/training-perpetrator-distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import torch
import numpy as np



# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")



GPU is available. Using: Tesla T4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Assuming your CSV file is named 'conflict_data.csv'
df = pd.read_csv('/content/drive/MyDrive/SATP_data/perpetrator.csv')

# Select only the necessary columns
df = df[['incident_summary', 'perpetrator']]

# Drop rows with missing values
df.dropna(inplace=True)

# Assuming your DataFrame is named 'df'
unique_perpetrators = df['perpetrator'].unique()

# Display the unique values
print("Unique values in 'perpetrator':")
print(unique_perpetrators)



Unique values in 'perpetrator':
['Security' 'Maoist' 'Unknown']


In [ ]:


# Verify the replacement
unique_perpetrators = df['perpetrator'].unique()
print("Updated unique values in 'perpetrator':")
print(unique_perpetrators)

Updated unique values in 'perpetrator':
['Security' 'Maoist' 'Unknown']


In [ ]:


# Split the data into train, test, and validation sets
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['perpetrator'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert the DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:


model_name = "distilbert-base-uncased" # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

def tokenize_function(example):
    return tokenizer(example['incident_summary'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'perpetrator'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'perpetrator'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'perpetrator'])


Map:   0%|          | 0/7931 [00:00<?, ? examples/s]

Map:   0%|          | 0/991 [00:00<?, ? examples/s]

Map:   0%|          | 0/992 [00:00<?, ? examples/s]

In [ ]:
# Map the 'perpetrator' to integer labels
label_map = {label: i for i, label in enumerate(df['perpetrator'].unique())}
train_dataset = train_dataset.map(lambda x: {'labels': label_map[x['perpetrator']]})
val_dataset = val_dataset.map(lambda x: {'labels': label_map[x['perpetrator']]})
test_dataset = test_dataset.map(lambda x: {'labels': label_map[x['perpetrator']]})


# Load pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_map))
model.to(device)


Map:   0%|          | 0/7931 [00:00<?, ? examples/s]

Map:   0%|          | 0/991 [00:00<?, ? examples/s]

Map:   0%|          | 0/992 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
label_map

{'Security': 0, 'Maoist': 1, 'Unknown': 2}

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Aligning save strategy with evaluation strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)


data_collator = DataCollatorWithPadding(tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,0.074500,0.140930
2,0.216800,0.132266
3,0.024800,0.134115


TrainOutput(global_step=1488, training_loss=0.16718622468291752, metrics={'train_runtime': 389.9649, 'train_samples_per_second': 61.013, 'train_steps_per_second': 3.816, 'total_flos': 1098249350589540.0, 'train_loss': 0.16718622468291752, 'epoch': 3.0})

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

# Load accuracy metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


In [ ]:
# Get raw predictions
predictions, labels, _ = trainer.predict(test_dataset)

# Calculate accuracy
accuracy = compute_metrics((predictions, labels))['accuracy']
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.9536290322580645


In [ ]:

from sklearn.metrics import classification_report

# Get predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Convert numerical labels back to original perpetrator names
reverse_label_map = {v: k for k, v in label_map.items()}
predicted_perpetrators = [reverse_label_map[label] for label in predicted_labels]
true_perpetrators = [reverse_label_map[label] for label in labels]

# Generate classification report
report = classification_report(true_perpetrators, predicted_perpetrators)
report

'              precision    recall  f1-score   support\n\n      Maoist       0.96      0.97      0.97       533\n    Security       0.96      0.94      0.95       423\n     Unknown       0.76      0.78      0.77        36\n\n    accuracy                           0.95       992\n   macro avg       0.89      0.90      0.90       992\nweighted avg       0.95      0.95      0.95       992\n'

              precision    recall  f1-score   support

      Maoist       0.96      0.97      0.97       533
    Security       0.96      0.94      0.95       423
     Unknown       0.76      0.78      0.77        36

    accuracy                           0.95       992
   macro avg       0.89      0.90      0.90       992
weighted avg       0.95      0.95      0.95       992


              precision    recall  f1-score   support

      Maoist       0.96      0.97      0.97       533
    Security       0.97      0.94      0.95       423
     Unknown       0.68      0.78      0.73        36

    accuracy                           0.95       992
   macro avg       0.87      0.90      0.88       992
weighted avg       0.95      0.95      0.95       992


In [ ]:
# prompt: save the model and tokenxer

trainer.save_model("/content/drive/MyDrive/SATP_data/saved_model")
tokenizer.save_pretrained("/content/drive/MyDrive/SATP_data/saved_tokenizer")

In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/MyDrive/SATP_data/perpetrator/distilBert')
tokenizer.save_pretrained('/content/drive/MyDrive/SATP_data/perpetrator/distilBert')


('/content/drive/MyDrive/SATP_data/perpetrator/distilBert/tokenizer_config.json',
 '/content/drive/MyDrive/SATP_data/perpetrator/distilBert/special_tokens_map.json',
 '/content/drive/MyDrive/SATP_data/perpetrator/distilBert/vocab.txt',
 '/content/drive/MyDrive/SATP_data/perpetrator/distilBert/added_tokens.json',
 '/content/drive/MyDrive/SATP_data/perpetrator/distilBert/tokenizer.json')